```python
# input_data shape
Input: (batch_size, in_channel, width, height)
# conv layer
class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
input: (Batch_size, C_in, H_in, W_in)
output: (Batch_size, C_out, H_out, W_out)

weight(tensor): (out_channels, in_channels,kernel_size)
bias(tensor): (out_channel)
```

In [37]:
from itertools import product

import time
import numpy as np
from sklearn import datasets
from scipy.stats import multivariate_normal as mvn
import matplotlib.pyplot as plt

from ripser import Rips
from persim import PersistenceImager

import os
import math

import seaborn
import pandas as pd
import torch
import random
from torch import nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import re
import cv2
from PIL import Image

In [38]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
# parameters
learning_rate = 0.001
training_epochs = 500
batchsize = 25


In [39]:
# dir = os.listdir("./metric")
# data = []
# a = []
# for metric in dir:
#     a = re.findall("\d+\.?\d*", metric)
#     data.append([np.loadtxt("./metric/"+metric), metric])
#     # print(a[-2])
#     print(metric[-2])


In [40]:
class My_dataset(Dataset):
    def __init__(self, train):
        super().__init__()
        # 使用sin函数返回10000个时间序列,如果不自己构造数据，就使用numpy,pandas等读取自己的数据为x即可。
        # 以下数据组织这块既可以放在init方法里，也可以放在getitem方法里
        dir = os.listdir("./grids_trainset_cifar")
        dirtest = os.listdir("./grids_testset_cifar")
        data = []
        a = []
        if train:
            # for metric in dir:
            #     data.append(
            #         [np.loadtxt("./grids_trainset_cifar/"+metric), metric])
            sample = []
            for metric in dir:
                sample.append([np.loadtxt("./grids_trainset_cifar/"+metric), metric])
                if len(sample) == 4:
                    res = cv2.merge([i[0] for i in sample])
                    res = np.transpose(res,(2,0,1))
                    data.append([res, metric])
                    sample = []
            
        else:
            # for metric in dirtest:
            #     data.append(
            #         [np.loadtxt("./grids_testset_cifar/"+metric), metric])
            sample = []
            for metric in dirtest:
                sample.append([np.loadtxt("./grids_testset_cifar/"+metric), metric])
                if len(sample) == 4:
                    res = cv2.merge([i[0] for i in sample])
                    res = np.transpose(res,(2,0,1))
                    data.append([res, metric])
                    sample = []

        self.x = [item[0] for item in data]
        self.y = [math.ceil(int(re.findall("\d+\.?\d*", item[1])[0]) /
                            (int(re.findall("\d+\.?\d*", item[1])[0])+1)) for item in data]
        # self.y = [int(re.findall("\d+\.?\d*", item[1])[0]) for item in data]
        self.src,  self.trg = [], []
        for i in range(len(data)):
            self.src.append(self.x[i])
            self.trg.append(self.y[i])

    def __getitem__(self, index):
        return self.src[index], self.trg[index]

    def __len__(self):
        return len(self.src)

 # 或者return len(self.trg), src和trg长度一样


In [41]:
data_tf = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])
     
data_train = My_dataset(train=True)
data_test = My_dataset(train=False)
data_loader_train = DataLoader(data_train, batch_size=batchsize, shuffle=True)
data_loader_test = DataLoader(data_test, batch_size=batchsize, shuffle=True)


# i_batch的多少根据batch size和def __len__(self)返回的长度确定
# batch_data返回的值根据def __getitem__(self, index)来确定
# 对训练集：(不太清楚enumerate返回什么的时候就多print试试)
# for i_batch, batch_data in enumerate(data_loader_train):
#     print(i_batch)  # 打印batch编号
#     print(batch_data[0].shape)  # 打印该batch里面src
#     print(batch_data[1])  # 打印该batch里面trg
# # # 对测试集：（下面的语句也可以）
# for i_batch, (src, trg) in enumerate(data_loader_test):
#     print(i_batch)  # 打印batch编号
#     print(src)  # 打印该batch里面src的尺寸
#     print(trg)  # 打印该batch里面trg的尺寸


In [42]:
# class MyNet(torch.nn.Module):
#     def __init__(self):
#         super(MyNet, self).__init__()

#         self.fc = torch.nn.Sequential(
#             torch.nn.Linear(256*256, 1024),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(1024, 1024),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(1024, 128),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(128, 64),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(64, 2)
#         )

#     def forward(self, x):
#         x = x.view(-1,256*256)
#         x = self.fc(x)
#         return x


# model = MyNet().to(device)

In [43]:
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(4, 16, kernel_size=64),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(inplace=True)
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(16 * 32 * 32, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(64, 2)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


model = MyNet().to(device)

In [44]:
# class MyNet(torch.nn.Module):
#     def __init__(self):
#         super(MyNet, self).__init__()
#         self.layer1 = torch.nn.Sequential(
#             torch.nn.Conv2d(4, 16, kernel_size=3),
#             torch.nn.BatchNorm2d(16),
#             torch.nn.ReLU(inplace=True)
#         )

#         self.layer2 = torch.nn.Sequential(
#             torch.nn.MaxPool2d(kernel_size=2, stride=2)
#         )

#         self.fc = torch.nn.Sequential(
#             torch.nn.Linear(16 * 63 * 63, 1024),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(1024, 1024),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(1024, 128),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(128, 64),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(64, 2)
#         )

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = x.view(x.size(0), -1)
#         x = self.fc(x)
#         return x


# model = MyNet().to(device)

In [45]:
# class MyNet(torch.nn.Module):
#     def __init__(self):
#         super(MyNet, self).__init__()
#         self.layer1 = torch.nn.Sequential(
#             torch.nn.Conv2d(4, 16, kernel_size=3),
#             torch.nn.BatchNorm2d(16),
#             torch.nn.ReLU(inplace=True)
#         )

#         self.layer2 = torch.nn.Sequential(
#             torch.nn.MaxPool2d(kernel_size=2, stride=2)
#         )

#         self.layer3 = torch.nn.Sequential(
#             torch.nn.Conv2d(16, 32, kernel_size=3),
#             torch.nn.BatchNorm2d(32),
#             torch.nn.ReLU(inplace=True)
#         )

#         self.layer4 = torch.nn.Sequential(
#             torch.nn.MaxPool2d(kernel_size=2, stride=2)
#         )

#         self.fc = torch.nn.Sequential(
#             torch.nn.Linear(32 * 30 * 30, 1024),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(1024, 1024),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(1024, 128),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(128, 64),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(64, 2)
#         )

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)
#         x = x.view(x.size(0), -1)
#         x = self.fc(x)
#         return x


# model = MyNet().to(device)
# # print(model)


In [46]:
# define cost/loss & optimizer
# Softmax is internally computed.
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [47]:
total_batch = len(data_loader_train)

for epoch in range(training_epochs):
    model.train()
    avg_cost = 0

    # for X, Y in data_loader_train:
    #     X = torch.autograd.Variable(X).to(device).float()
    #     Y = Y.to(device)

    #     optimizer.zero_grad()
    #     hypothesis = model(X)
    #     cost = criterion(hypothesis, Y)
    #     cost.backward()
    #     optimizer.step()

    #     avg_cost += cost / total_batch
    # model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(data_loader_train):
        inputs, targets = inputs.to(device).float(), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        avg_cost += loss / total_batch

    
    # if avg_cost < 0.1:
    #     break
    model.eval()
    # with torch.no_grad():
    #     for X_test, Y_test in data_loader_test:
    #         X_test = X_test.to(device).float()
    #         Y_test = Y_test.to(device)

    #     prediction = model(X_test)
    #     correct_prediction = torch.argmax(prediction, 1) == Y_test
    #     accuracy = correct_prediction.float().mean()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader_train):
            inputs, targets = inputs.to(device).float(), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    trainacc = 100.*correct/total
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader_test):
            inputs, targets = inputs.to(device).float(), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    testacc = 100.*correct/total
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 'trainacc:', trainacc, 'testacc:', testacc)

print('Learning finished')


Epoch: 0001 cost = 0.694497466 trainacc: 50.53763440860215 testacc: 53.84615384615385
Epoch: 0002 cost = 0.694859803 trainacc: 50.53763440860215 testacc: 53.84615384615385
Epoch: 0003 cost = 0.695838511 trainacc: 50.53763440860215 testacc: 53.84615384615385
Epoch: 0004 cost = 0.693006337 trainacc: 50.53763440860215 testacc: 53.84615384615385
Epoch: 0005 cost = 0.695089340 trainacc: 50.53763440860215 testacc: 53.84615384615385
Epoch: 0006 cost = 0.693205476 trainacc: 50.53763440860215 testacc: 53.84615384615385
Epoch: 0007 cost = 0.692490339 trainacc: 50.53763440860215 testacc: 53.84615384615385
Epoch: 0008 cost = 0.691832066 trainacc: 50.53763440860215 testacc: 53.84615384615385
Epoch: 0009 cost = 0.690589845 trainacc: 50.53763440860215 testacc: 53.84615384615385
Epoch: 0010 cost = 0.692083538 trainacc: 50.53763440860215 testacc: 53.84615384615385
Epoch: 0011 cost = 0.692712367 trainacc: 50.53763440860215 testacc: 53.84615384615385
Epoch: 0012 cost = 0.691909671 trainacc: 50.5376344086

In [48]:
# total_batch = len(data_loader_train)
# model.train()
# for epoch in range(training_epochs):
#     avg_cost = 0

#     for X, Y in data_loader_train:
#         X = torch.autograd.Variable(X).to(device).float()
#         Y = Y.to(device)

#         optimizer.zero_grad()
#         hypothesis = model(X)
#         cost = criterion(hypothesis, Y)
#         cost.backward()
#         optimizer.step()

#         avg_cost += cost / total_batch

#     print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

# print('Learning finished')


In [49]:
# Test the model using test sets
# model.load_state_dict(torch.load(".\merge_save\merge_attack0normal10_05-07--21-10-32.pth")["state_dict"])
# model.load_state_dict(torch.load(r".\myfed_normal_save\model18-24-02.pth")["state_dict"])
model.eval()
with torch.no_grad():
    for X_test, Y_test in data_loader_test:
        X_test = X_test.to(device).float()
        Y_test = Y_test.to(device)

        prediction = model(X_test)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        print(prediction,correct_prediction, Y_test)
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

    # Get one and predict
    # r = random.randint(0, len(mnist_test) - 1)
    # X_single_data = mnist_test.data[r:r + 1].view(-1, 28 * 28).float().to(device)
    # Y_single_data = mnist_test.targets[r:r + 1].to(device)

    # print('Label: ', Y_single_data.item())
    # single_prediction = model(X_single_data)
    # print('Prediction: ', torch.argmax(single_prediction, 1).item())


tensor([[ 1.1895e-01, -2.2535e-01],
        [ 1.2109e-01, -2.1165e-01],
        [-2.7388e-01,  2.3258e-01],
        [ 1.2152e-01, -2.8349e-01],
        [-1.8620e-01,  1.0789e-01],
        [ 1.6650e-01, -2.8374e-01],
        [-5.3804e-02, -4.0294e-02],
        [ 7.8285e-02, -2.0636e-01],
        [ 1.8691e-01, -3.1492e-01],
        [ 2.2518e-04, -9.1857e-02],
        [-6.8734e-03, -1.0427e-01],
        [-2.4301e-01,  1.9451e-01],
        [-7.0770e-02, -4.8966e-02],
        [-3.5307e-01,  3.0423e-01],
        [ 3.1977e-02, -1.5208e-01],
        [-1.7466e-03, -7.7578e-02],
        [-2.1794e-01,  1.2970e-01],
        [-1.5280e-01,  2.2714e-02],
        [-3.1461e-01,  2.7028e-01],
        [ 6.5254e-02, -1.8216e-01],
        [-2.5813e-01,  1.9621e-01],
        [-2.7304e-01,  2.1858e-01],
        [ 1.3397e-01, -2.4446e-01],
        [ 2.0213e-01, -3.2848e-01],
        [-4.9636e-02,  5.0565e-04]], device='cuda:0') tensor([False,  True,  True,  True,  True,  True, False, False,  True, False,
    